In [1]:
'''Figures showing UQ for impact site'''
import numpy as np
import pandas as pd
from GPR import *



'''plots the prediction in red, and plots the '''
def plot_impact_site_with_uncertainty(x_pred, x_std, y_pred, y_std, x_true, y_true, saving_path=None):
    #material basis vectors for RPA bone
    Material_X = np.array([-0.87491124, -0.44839274,  0.18295974])
    Material_Y = np.array([ 0.23213791, -0.71986519, -0.65414532])
    Material_Z = np.array([ 0.42502036, -0.5298472,   0.7339071 ])
    #Center of mass of the RPA bone in abaqus basis
    CM = np.array([106.55,72.79,56.64])
    # #Ossification center of the RPA bone in abaqus basis
    OC = np.array([130.395996,46.6063,98.649696])
    
    parietal_node_location_df = pd.read_csv('/Users/jakehirst/Desktop/sfx/sfx_ML_code/sfx_ML/Feature_gathering/parital_node_locations.csv')
    RPA_x = parietal_node_location_df['RPA nodes x']; RPA_y = parietal_node_location_df['RPA nodes y']; RPA_z = parietal_node_location_df['RPA nodes z']
    
    #converting the RPA node locations into Jimmy's reference frame
    RPA_x, RPA_y, RPA_z = convert_coordinates_to_new_basis(Material_X, Material_Y, Material_Z, CM, RPA_x, RPA_y, RPA_z)
    
    #loading previously trained models
    # model_x = load_GPR_model(f'/Users/jakehirst/Desktop/model_results/MODEL_COMPARISONS/GPR_{labels_to_predict[0]}/GPR_model_fold{models_fold_to_pull[labels_to_predict[0]]}.sav')
    # model_y = load_GPR_model(f'/Users/jakehirst/Desktop/model_results/MODEL_COMPARISONS/GPR_{labels_to_predict[1]}/GPR_model_fold{models_fold_to_pull[labels_to_predict[1]]}.sav')
    # model_z = load_GPR_model(f'/Users/jakehirst/Desktop/model_results/GPR_{labels_to_predict[2]}/GPR_model_fold{models_fold_to_pull[labels_to_predict[2]]}.sav')
    
    #predicting with previously trained models  
    # x_predictions, x_stds = model_x.predict(full_dataset[all_important_features[labels_to_predict[0]]].to_numpy(), return_std=True)
    # y_predictions, y_stds = model_y.predict(full_dataset[all_important_features[labels_to_predict[1]]].to_numpy(), return_std=True)
    # z_predictions, z_stds = model_z.predict(remove_ABAQUS_features(full_dataset[all_important_features[labels_to_predict[2]]]).to_numpy(), return_std=True)
    
    # x_true = full_dataset[labels_to_predict[0]].to_numpy()
    # y_true = full_dataset[labels_to_predict[1]].to_numpy()
    # z_true = full_dataset['impact site z'].to_numpy()
    
    '''3d plot of RPA nodes and the predicted x, y and z values in '''
    # Create a 3D figure
    fig = plt.figure(figsize=(10,8))
    ax = fig.add_subplot(111, projection='3d')
    ax.grid(False)
    
    ax.scatter(RPA_z, RPA_x, RPA_y, c='grey', alpha=0.025)
    # x_dist = np.random.normal(x_predictions[i], x_stds[i], 100)
    # y_dist = np.random.normal(y_predictions[i], y_stds[i], 100)
    x_dist = np.random.normal(x_pred, x_std, 100)
    y_dist = np.random.normal(y_pred, y_std, 100)
    
    point_size = 50
    
    z_val=15
    z_dist = np.random.normal(z_val, 3, 5000) #TODO find a good value for z
    ax.scatter(z_dist, x_dist, y_dist, c='cyan', alpha=0.01)
    ax.scatter(z_val, x_pred, y_pred, c='blue', label='Mean predicted impact location', s=point_size)
    ax.scatter(z_val, x_true, y_true, c='orange', label='True impact location', s=point_size)
    

    ax.set_xlabel('Z')
    ax.set_ylabel('X')
    ax.set_zlabel('Y')
    ax.set_xlim3d(-60, 60)
    ax.set_ylim3d(-80, 80)
    ax.set_zlim3d(-60, 60)
    ax.set_title('GPR prediction for impact site in right parietal bone', fontweight='bold')
    ax.legend()
    
    normal_vector = np.array([0,1,0])
    # Calculate the azimuthal and polar angles
    azimuth = np.arctan2(normal_vector[1], normal_vector[0])
    polar = np.arccos(normal_vector[2])
    # Convert angles to degrees
    azimuth = np.degrees(azimuth)
    polar = np.degrees(polar)
    # # Set the camera direction using the angles (customizing a bit)
    ax.view_init(elev=-5, azim=azimuth + 270)
    # # Show the plot
    if(saving_path == None):
        plt.show()
    else:
        plt.savefig(saving_path)
    plt.close()
    return




model_types = ['Single RF', 'RF fed GPR']
fold = 1

# parietal_nodes_folder = ''

for model_type in model_types:
    for fold in range(1,6):
        x_model_folder = f'/Volumes/Jake_ssd/Paper 2/without_transformations/UQ_bagging_models_without_transformations/impact site x/{model_type}'
        y_model_folder = f'/Volumes/Jake_ssd/Paper 2/without_transformations/UQ_bagging_models_without_transformations/impact site y/{model_type}'

        #defining folders to get the models and to store the results
        model_saving_folder = f'{model_folder}/{label_to_predict}/{model_type}/{num_models}_models/fold_{fold_no}'
        results_saving_folder = f'{results_folder}/{label_to_predict}/{model_type}/{num_models}_models/fold_{fold_no}'
        
        #defining folders where the datasets are coming from (5-fold cv)
        test_features_path = Paper2_path + f'/5fold_datasets/{label_to_predict}/fold{fold_no}/test_features.csv'
        test_labels_path = Paper2_path + f'/5fold_datasets/{label_to_predict}/fold{fold_no}/test_labels.csv'
        train_features_path = Paper2_path + f'/5fold_datasets/{label_to_predict}/fold{fold_no}/train_features.csv'
        train_labels_path = Paper2_path + f'/5fold_datasets/{label_to_predict}/fold{fold_no}/train_labels.csv'

        #defining the features that each model used (since they vary with each model)
        features_to_keep = ????
        
        if(model_type in ['ANN', 'RF', 'GPR','ridge']):
            #defining folders to get the models and to store the results
            model_saving_folder = f'{model_folder}/{label_to_predict}/{model_type}/{num_models}_models/fold_{fold_no}'
            results_saving_folder = f'{results_folder}/{label_to_predict}/{model_type}/{num_models}_models/fold_{fold_no}'
            #predicting the test and train sets with the bagging models
            test_r2, test_ensemble_predictions, test_uncertanties, test_labels = Get_predictions_and_uncertainty_with_bagging(test_features_path, test_labels_path, model_saving_folder, results_saving_folder, features_to_keep, label_to_predict, model_type)
            train_r2, train_ensemble_predictions, train_uncertanties, train_labels = Get_predictions_and_uncertainty_with_bagging(train_features_path, train_labels_path, model_saving_folder, results_saving_folder, features_to_keep, label_to_predict, model_type)
        else:
            #defining folders to get the models and to store the results
            model_saving_folder = f'{model_folder}/{label_to_predict}/{model_type}/1_models/fold_{fold_no}'
            results_saving_folder = f'{results_folder}/{label_to_predict}/{model_type}/1_models/fold_{fold_no}'
            #predicting the test and train sets with the NON bagging models
            test_r2, test_ensemble_predictions, test_uncertanties, test_labels = Get_predictions_and_uncertainty(test_features_path, test_labels_path, model_saving_folder, results_saving_folder, features_to_keep, label_to_predict, model_type)
            train_r2, train_ensemble_predictions, train_uncertanties, train_labels = Get_predictions_and_uncertainty(train_features_path, train_labels_path, model_saving_folder, results_saving_folder, features_to_keep, label_to_predict, model_type)



    


SyntaxError: incomplete input (1850306633.py, line 44)